# BentoML Example: Fast AI with Tabular data

**BentoML makes moving trained ML models to production easy:**

* Package models trained with **any ML framework** and reproduce them for model serving in production
* **Deploy anywhere** for online API serving or offline batch serving
* High-Performance API model server with *adaptive micro-batching* support
* Central hub for managing models and deployment process via Web UI and APIs
* Modular and flexible design making it *adaptable to your infrastrcuture*

BentoML is a framework for serving, managing, and deploying machine learning models. It is aiming to bridge the gap between Data Science and DevOps, and enable teams to deliver prediction services in a fast, repeatable, and scalable way.
Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.


This notebook is based on fastai v1's cours v3 lesson 4.  It will train a model that predict salary range base on the data we provided.


![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=fast-ai&ea=fast-ai-salary-range-prediction&dt=fast-ai-salary-range-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install -q -U 'fastai<=1.0.61'

In [3]:
from fastai.tabular import *

## Prepare Training Data

In [4]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [5]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [6]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [7]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [8]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Federal-gov,HS-grad,Widowed,Other-service,Not-in-family,White,False,2.5957,-0.0779,-0.4224,<50k
Federal-gov,11th,Divorced,Adm-clerical,Not-in-family,White,False,2.4491,-1.4279,-1.2046,<50k
?,HS-grad,Married-civ-spouse,?,Husband,White,False,1.6428,0.0121,-0.4224,<50k
Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,False,-0.0430,-0.2710,-0.0312,>=50k
Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Black,False,0.1036,0.3476,-0.4224,<50k
Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,False,0.2502,-0.6640,-0.4224,<50k
Private,HS-grad,Never-married,Other-service,Not-in-family,Black,False,-0.9959,0.0384,-0.4224,<50k
Private,Some-college,Never-married,Other-service,Not-in-family,White,False,-1.2891,0.3968,-0.0312,<50k
?,Some-college,Never-married,?,Own-child,White,False,-1.3624,-1.4083,-0.0312,<50k
Private,Bachelors,Never-married,Sales,Not-in-family,White,False,-0.6294,-0.6502,1.1422,>=50k


## Model Training

In [9]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [10]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.351867,0.388441,0.825000,00:05


In [11]:
row = df.iloc[0] # sample input date for testing

learn.predict(row)

(Category tensor(1), tensor(1), tensor([0.4563, 0.5437]))

In [12]:
df.iloc[0].T

age                                49
workclass                     Private
fnlwgt                         101320
education                  Assoc-acdm
education-num                    12.0
marital-status     Married-civ-spouse
occupation                        NaN
relationship                     Wife
race                            White
sex                            Female
capital-gain                        0
capital-loss                     1902
hours-per-week                     40
native-country          United-States
salary                          >=50k
Name: 0, dtype: object

## Create BentoService for model serving

In [13]:
%%writefile tabular_csv.py

from bentoml import env, api, artifacts, BentoService
from bentoml.frameworks.fastai import Fastai1ModelArtifact
from bentoml.adapters import DataframeInput


@env(pip_packages=['fastai'])
@artifacts([Fastai1ModelArtifact('model')])
class FastaiTabularModel(BentoService):
    
    @api(input=DataframeInput(), batch=True)
    def predict(self, df):
        results = []
        for _, row in df.iterrows():       
            prediction = self.artifacts.model.predict(row)
            results.append(prediction[0].obj)
        return results

Writing tabular_csv.py


## Save BentoService to file archive

In [14]:
# 1) import the custom BentoService defined above
from tabular_csv import FastaiTabularModel

# 2) `pack` it with required artifacts
svc = FastaiTabularModel()
svc.pack('model', learn)

# 3) save your BentoSerivce
saved_path = svc.save()

[2021-03-11 15:32:56,972] INFO - Detected zipimporter <zipimporter object "/home/ec2-user/anaconda3/lib/python3.7/site-packages/locket-0.2.1-py3.7.egg">
[2021-03-11 15:32:56,976] WARNING - BentoML by default does not include spacy and torchvision package when using FastaiModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
[2021-03-11 15:32:56,978] WARNING - Overwriting existing pip package requirement 'fastai==1.0.61' to 'fastai<2.0.0'


/home/ec2-user/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[2021-03-11 15:32:57,857] INFO - BentoService bundle 'FastaiTabularModel:20210311153257_2D1346' saved to: /home/ec2-user/bentoml/repository/FastaiTabularModel/20210311153257_2D1346


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [ ]:
!bentoml serve FastaiTabularModel:latest

/home/ec2-user/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2021-03-11 15:33:07,972] INFO - Getting latest version FastaiTabularModel:20210311153257_2D1346
[2021-03-11 15:33:07,972] INFO - Starting BentoML API server in development mode..
[2021-03-11 15:33:08,240] INFO - Detected zipimporter <zipimporter object "/home/ec2-user/bentoml/repository/FastaiTabularModel/20210311153257_2D1346/FastaiTabularModel/zipimports/locket-0.2.1-py3.7.egg">
[2021-03-11 15:33:09,737] WARNING - BentoML by default does not include spacy and torchvision package when using FastaiModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serve FastaiTabularModel:latest --run-with-ngrok

### Send prediction requeset to the REST API server

#### JSON Request

```bash
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '[{
  "age": 49,
  "workclass": "Private",
  "fnlwgt": 101320,
  "education": "Assoc-acdm",
  "education-num": 12.0,
  "marital-status": "Married-civ-spouse",
  "occupation": "",
  "relationship": "Wift",
  "race": "White",
  "sex": "Female",
  "capital-gain": 0,
  "capital-loss": 1902,
  "hours-per-week": 40,
  "native-country": "United-States",
  "salary": ">=50k"
}]'
```

#### CSV Request

```bash
curl -X POST "http://127.0.0.1:5000/predict" \
    -H "Content-Type: text/csv" \
    --data-binary @test.csv
```

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [16]:
!bentoml containerize FastaiTabularModel:latest

/home/ec2-user/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2021-03-11 16:46:28,489] INFO - Getting latest version FastaiTabularModel:20210311153257_2D1346
Found Bento: /home/ec2-user/bentoml/repository/FastaiTabularModel/20210311153257_2D1346
Containerizing FastaiTabularModel:20210311153257_2D1346 with local YataiService and docker daemon from local environment|Build container image: fastaitabularmodel:20210311153257_2D1346


In [20]:
FastaiTabularModel


tabular_csv.FastaiTabularModel

In [22]:
!docker info

Client:
 Debug Mode: false

Server:
 Containers: 31
  Running: 0
  Paused: 0
  Stopped: 31
 Images: 37
 Server Version: 19.03.13-ce
 Storage Driver: overlay2
  Backing Filesystem: xfs
  Supports d_type: true
  Native Overlay Diff: true
 Logging Driver: json-file
 Cgroup Driver: cgroupfs
 Plugins:
  Volume: local
  Network: bridge host ipvlan macvlan null overlay
  Log: awslogs fluentd gcplogs gelf journald json-file local logentries splunk syslog
 Swarm: inactive
 Runtimes: neuron nvidia runc
 Default Runtime: runc
 Init Binary: docker-init
 containerd version: c623d1b36f09f8ef6536a057bd658b3aa8632828
 runc version: ff819c7e9184c13b7c2607fe6c30ae19403a7aff
 init version: de40ad0 (expected: fec3683)
 Security Options:
  seccomp
   Profile: default
 Kernel Version: 4.14.219-161.340.amzn2.x86_64
 Operating System: Amazon Linux 2
 OSType: linux
 Architecture: x86_64
 CPUs: 4
 Total Memory: 7.296GiB
 Name: ip-172-31-1-118.us-east-2.compute.internal
 ID: ULQY:BXOT:K6WH:QM6E:IDFQ:2EML:NOQB:UQ

In [23]:
!docker run -p 5000:5000 fastaitabularmodel:20210311153257_2D1346

[2021-03-11 16:59:44,222] INFO - Starting BentoML API server in production mode..
[2021-03-11 16:59:44,374] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2021-03-11 16:59:44 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2021-03-11 16:59:44 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2021-03-11 16:59:44 +0000] [1] [INFO] Using worker: sync
[2021-03-11 16:59:44 +0000] [11] [INFO] Booting worker with pid: 11
[2021-03-11 16:59:44 +0000] [12] [INFO] Booting worker with pid: 12
[2021-03-11 16:59:44 +0000] [13] [INFO] Booting worker with pid: 13
[2021-03-11 16:59:44,628] WARNING - Saved BentoService Python version mismatch: loading BentoService bundle created with Python version 3.7.10, but current environment version is 3.7.6.
[2021-03-11 16:59:44,640] INFO - Detected zipimporter <zipimporter object "/bento/FastaiTabularModel/zipimports/locket-0.2.1-py3.7.egg">
[2021-03-11 16:59:44,705] WARNING - Saved BentoService Python version mismatch: loading BentoService bu

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [18]:
saved_path

'/home/ec2-user/bentoml/repository/FastaiTabularModel/20210311153257_2D1346'

In [31]:
from bentoml import load

svc = load(saved_path)
print(svc.predict(df.iloc[0:1]))

[2021-03-11 17:09:47,350] WARNING - Module `tabular_csv` already loaded, using existing imported module.
[2021-03-11 17:09:47,357] WARNING - pip package requirement pandas already exist
[2021-03-11 17:09:47,357] WARNING - BentoML by default does not include spacy and torchvision package when using FastaiModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
[2021-03-11 17:09:47,358] WARNING - pip package requirement torch already exist
[2021-03-11 17:09:47,360] WARNING - pip package requirement fastai<2.0.0 already exist
['>=50k']


In [29]:
svc.predict(df.iloc[0:1])

['>=50k']

## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [ ]:
!wget https://raw.githubusercontent.com/bentoml/gallery/master/fast-ai/salary-range-prediction/test.csv
!bentoml run FastaiTabularModel:latest predict \
    --input-file test.csv

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)

